<a href="https://colab.research.google.com/github/KeyurIITGN/TEMP_REP/blob/main/code_smai_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probability Tutorial

Course: Statistical Methods in AI

Date: 3 February, 2025

## 0. Setup

In [ ]:
%pip install matplotlib numpy opencv-python scipy statsmodels

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import binom, chi2, expon, kstest, norm, poisson
from statsmodels.sandbox.stats.runs import runstest_1samp

## 1. Probability Foundations

### 1.1 Pseudo-Randomness

#### 1.1.1 Linear Congruential Generator

Adapted from [The Art of Computer Programming: Seminumerical Algorithms](https://www.amazon.in/Art-Computer-Programming-Seminumerical-Algorithms-ebook/dp/B01AY4ZHKE) (Chapter 3).

In [ ]:
# Generator
def lcg(n, m = 10, a = 7, c = 7, seed = 7):
    x = seed
    sequence = []
    for _ in range(n):
        x = (a * x + c) % m
        sequence.append(x)
    return sequence

# Sequence
sequence = lcg(10)
plt.hist(sequence, bins=10)
plt.title("Histogram of LCG Sequence")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

# 1. Runs Test
runs_test_statistic, runs_test_p_value = runstest_1samp(sequence)
print(f"Runs Test - Statistic: {runs_test_statistic}, P-value: {runs_test_p_value}, Passed: {runs_test_p_value >= 0.05}")

# 2. Chi-Square Test
observed_frequencies = np.histogram(sequence, bins=10, range=(0, 9))[0].astype(float)
expected_frequencies = np.full_like(observed_frequencies, len(sequence) / 10)
chi_square_statistic = np.sum((observed_frequencies - expected_frequencies)**2 / expected_frequencies)
chi_square_p_value = 1 - chi2.cdf(chi_square_statistic, df=9)
print(f"Chi-Square Test - Statistic: {chi_square_statistic}, P-value: {chi_square_p_value}, Passed: {chi_square_p_value >= 0.05}")

# 3. Kolmogorov-Smirnov Test
ks_statistic, ks_p_value = kstest(sequence, 'uniform')
print(f"Kolmogorov-Smirnov Test - Statistic: {ks_statistic}, P-value: {ks_p_value}, Passed: {ks_p_value >= 0.05}")

#### 1.1.2 Appreciating XOR

Adapted from [Khan Academy](https://www.khanacademy.org/computing/computer-science/cryptography/ciphers/a/xor-and-the-one-time-pad).

In [ ]:
image = cv2.imread('figures/babbage.jpg', cv2.IMREAD_GRAYSCALE)
random_bits = np.random.randint(0, 256, image.shape, dtype=np.uint8)
and_result = cv2.bitwise_and(image, random_bits)
or_result = cv2.bitwise_or(image, random_bits)
xor_result = cv2.bitwise_xor(image, random_bits)

fig = plt.figure(figsize=(15, 3))
fig.suptitle('Effect of different bitwise operations on an image')

plt.subplot(1, 5, 1)
plt.imshow(image, cmap='gray')
plt.title('Image')
plt.axis('off')

plt.subplot(1, 5, 2)
plt.imshow(random_bits, cmap='gray')
plt.title('Noise = U[0, 255]')
plt.axis('off')

plt.subplot(1, 5, 3)
plt.imshow(and_result, cmap='gray')
plt.title('AND(Image, Noise)')
plt.axis('off')

plt.subplot(1, 5, 4)
plt.imshow(or_result, cmap='gray')
plt.title('OR(Image, Noise)')
plt.axis('off')

plt.subplot(1, 5, 5)
plt.imshow(xor_result, cmap='gray')
plt.title('XOR(Image, Noise)')
plt.axis('off')

plt.tight_layout()
plt.show()

### 1.2 Useful Inequalities

#### 1.2.1 Markov's Inequality

In [ ]:
def markov_bound(mean, a):
    return mean / a

def binomial_actual_prob(n, p, a):
    return 1 - binom.cdf(a, n, p)

def exponential_actual_prob(scale, a):
    return 1 - expon.cdf(a, scale=scale)

def poisson_actual_prob(mu, a):
    return 1 - poisson.cdf(a, mu)

n = 50
p = 0.1
scale = 2.0
mu = 3.0

a_values = np.linspace(1, 20, 100)

binomial_bounds = [markov_bound(n * p, a) for a in a_values]
binomial_actual = [binomial_actual_prob(n, p, a) for a in a_values]

exponential_bounds = [markov_bound(scale, a) for a in a_values]
exponential_actual = [exponential_actual_prob(scale, a) for a in a_values]

poisson_bounds = [markov_bound(mu, a) for a in a_values]
poisson_actual = [poisson_actual_prob(mu, a) for a in a_values]

fig, axs = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Comparision of Markov bounds against probabilities computed using CDF')

axs[0].plot(a_values, binomial_bounds, label='Markov Bound', color='red')
axs[0].plot(a_values, binomial_actual, label='Actual Probability', color='blue')
axs[0].set_title('Binomial Distribution')
axs[0].set_xlabel('a')
axs[0].set_ylabel('Probability')
axs[0].legend()
axs[0].grid()

axs[1].plot(a_values, exponential_bounds, label='Markov Bound', color='red')
axs[1].plot(a_values, exponential_actual, label='Actual Probability', color='blue')
axs[1].set_title('Exponential Distribution')
axs[1].set_xlabel('a')
axs[1].set_ylabel('Probability')
axs[1].legend()
axs[1].grid()

axs[2].plot(a_values, poisson_bounds, label='Markov Bound', color='red')
axs[2].plot(a_values, poisson_actual, label='Actual Probability', color='blue')
axs[2].set_title('Poisson Distribution')
axs[2].set_xlabel('a')
axs[2].set_ylabel('Probability')
axs[2].legend()
axs[2].grid()

plt.tight_layout()
plt.show()

#### 1.2.2 Chebyshev's Inequality

In [ ]:
def gaussian_actual_probs(k):
    return 2 * (1 - norm.cdf(k))

def exponential_actual_probs(k):
    prob = np.zeros_like(k)
    mask = k < 1
    prob[mask] = (1 - np.exp(-(1 - k[mask]))) + np.exp(-(1 + k[mask]))
    prob[~mask] = np.exp(-(1 + k[~mask]))
    return prob

def uniform_actual_probs(k):
    return np.where(k <= np.sqrt(3), 1 - (k / np.sqrt(3)), 0)

k_values = np.linspace(1, 3, 100)
chebyshev_bounds = 1 / (k_values**2)

gaussian_probs = gaussian_actual_probs(k_values)
exponential_probs = exponential_actual_probs(k_values)
uniform_probs = uniform_actual_probs(k_values)

fig, axs = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Comparision of Chebyshev bounds against probabilities computed using CDF')

axs[0].plot(k_values, chebyshev_bounds, label='Chebyshev Bound', color='blue')
axs[0].plot(k_values, gaussian_probs, label='Actual Probability (Gaussian)', color='red')
axs[0].set_xlabel('k')
axs[0].set_ylabel('Probability')
axs[0].set_title('Gaussian Distribution')
axs[0].legend()
axs[0].grid(True)

axs[1].plot(k_values, chebyshev_bounds, label='Chebyshev Bound', color='blue')
axs[1].plot(k_values, exponential_probs, label='Actual Probability (Exponential)', color='green')
axs[1].set_xlabel('k')
axs[1].set_ylabel('Probability')
axs[1].set_title('Exponential Distribution')
axs[1].legend()
axs[1].grid(True)

axs[2].plot(k_values, chebyshev_bounds, label='Chebyshev Bound', color='blue')
axs[2].plot(k_values, uniform_probs, label='Actual Probability (Uniform)', color='purple')
axs[2].set_xlabel('k')
axs[2].set_ylabel('Probability')
axs[2].set_title('Uniform Distribution')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()

## 2. Probability Distributions

### 2.1 Discrete Probability Distributions

1. Visit https://www.desmos.com/calculator/gyl3z0aptn

2. Open the groups corresponding to different distributions in the sidebar.

3. Toggle the distributions you want to visualize.

4. Change the parameters of the distribution, initialized in the last group.

### 2.2 Continuous Probability Distributions

1. Visit https://www.desmos.com/calculator/vwxz5ldzov

2. Open the groups corresponding to different distributions in the sidebar.

3. Toggle the distributions you want to visualize.

### 2.3 Sampling from Distributions

In [ ]:
def inverse_transform_exponential(size, rate=1.0):
    u = np.random.uniform(0, 1, size)
    return -np.log(1 - u) / rate

def box_muller_normal(size):
    u1 = np.random.uniform(0, 1, size // 2)
    u2 = np.random.uniform(0, 1, size // 2)
    z0 = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
    z1 = np.sqrt(-2 * np.log(u1)) * np.sin(2 * np.pi * u2)
    return np.concatenate([z0, z1])

def acceptance_rejection_poisson(size, lambda_poisson):
    samples = []
    n = 100
    p = lambda_poisson / n
    while len(samples) < size:
        u = np.random.uniform(0, 1)
        y = np.random.binomial(n, p)
        if u <= (poisson.pmf(y, lambda_poisson) / binom.pmf(y, n, p)):
            samples.append(y)
    return np.array(samples)

num_samples = 10000
lambda_poisson = 5

exp_samples = inverse_transform_exponential(num_samples)
normal_samples = box_muller_normal(num_samples)
poisson_samples = acceptance_rejection_poisson(num_samples, lambda_poisson)

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Histogram (normalized) of generated samples against actual distribution')

x = np.linspace(0, 10, 1000)
axs[0].hist(exp_samples, bins=50, density=True, color='blue', alpha=0.7)
axs[0].plot(x, expon.pdf(x), color='red', label='Exponential PDF')
axs[0].set_title('Exponential Samples (Inverse Transform)')
axs[0].legend()

x = np.linspace(-4, 4, 1000)
axs[1].hist(normal_samples, bins=50, density=True, color='green', alpha=0.7)
axs[1].plot(x, norm.pdf(x), color='red', label='Normal PDF')
axs[1].set_title('Normal Samples (Box Muller)')
axs[1].legend()

x = np.arange(0, 20)
axs[2].hist(poisson_samples, bins=range(0, 20), density=True, color='orange', alpha=0.7, align='left')
axs[2].plot(x, poisson.pmf(x, lambda_poisson), 'ro-', label='Poisson PMF')
axs[2].set_title('Poisson Samples (Rejection Sampling)')
axs[2].legend()

plt.tight_layout()
plt.show()

ks_exp = kstest(exp_samples, 'expon')
print(f"KS Test for Exponential Samples: Statistic={ks_exp.statistic}, p-value={ks_exp.pvalue}, Passed: {ks_exp.pvalue >= 0.05}")

ks_norm = kstest(normal_samples, 'norm')
print(f"KS Test for Normal Samples: Statistic={ks_norm.statistic}, p-value={ks_norm.pvalue}, Passed: {ks_norm.pvalue >= 0.05}")

ks_poisson = kstest(poisson_samples, 'poisson', args=(lambda_poisson, 0))
print(f"KS Test for Poisson Samples: Statistic={ks_poisson.statistic}, p-value={ks_poisson.pvalue}, Passed: {ks_poisson.pvalue >= 0.05}")

## 3. Stochastic Processes

### 3.1 Markov Chains

#### 3.1.1 Limiting Distribution

In [ ]:
P = np.array([
    [0.5, 0.3, 0.2],
    [0.2, 0.6, 0.2],
    [0.3, 0.3, 0.4]
])
initial_dist = np.array([1.0, 0.0, 0.0])
print(f"Initial Distribution: {initial_dist}")

num_steps = 15
curr_dist = initial_dist
for i in range(num_steps):
    curr_dist = curr_dist @ P
    print(f"Step {i + 1}: {curr_dist}")

#### 3.1.2 BONUS: Queueing Networks

Look at Jackson Network which is modeled using a stationary routing matrix: https://en.wikipedia.org/wiki/Jackson_network.

You can look at the slides and Python simulation here: https://github.com/ihsingh2/open-queueing-networks/

### 3.2 Convergence

#### 3.2.1 Law of Large Numbers

In [ ]:
def running_average(data):
    return np.cumsum(data) / np.arange(1, len(data) + 1)

seq_len = 800

n = 50
p = 0.5
lambda_ = 5

binomial_samples = np.random.binomial(n, p, seq_len)
geometric_samples = np.random.geometric(p, seq_len)
poisson_samples = np.random.poisson(lambda_, seq_len)

binomial_running_avg = running_average(binomial_samples)
geometric_running_avg = running_average(geometric_samples)
poisson_running_avg = running_average(poisson_samples)

binomial_expected = n * p
geometric_expected = 1 / p
poisson_expected = lambda_

fig, axs = plt.subplots(1, 3, figsize=(16, 4))
fig.suptitle('Convergence of sample mean with increasing sample size')

axs[0].plot(binomial_running_avg)
axs[0].axhline(binomial_expected, color='r', linestyle='--')
axs[0].set_title('Binomial Distribution')

axs[1].plot(geometric_running_avg)
axs[1].axhline(geometric_expected, color='r', linestyle='--')
axs[1].set_title('Geometric Distribution')

axs[2].plot(poisson_running_avg)
axs[2].axhline(poisson_expected, color='r', linestyle='--')
axs[2].set_title('Poisson Distribution')

plt.tight_layout()
plt.show()

#### 3.2.2 Central Limit Theorem

In [ ]:
def normalize_means(means, population_mean, population_std):
    return (means - population_mean) / (population_std / np.sqrt(len(means)))

num_samples = 1000
sample_size = 1000

n = 50
p = 0.5
lambda_ = 5

binomial_samples = np.random.binomial(n, p, (num_samples, sample_size))
geometric_samples = np.random.geometric(p, (num_samples, sample_size))
poisson_samples = np.random.poisson(lambda_, (num_samples, sample_size))

binomial_means = np.mean(binomial_samples, axis=1)
geometric_means = np.mean(geometric_samples, axis=1)
poisson_means = np.mean(poisson_samples, axis=1)

binomial_normalized = normalize_means(binomial_means, n * p, np.sqrt(n * p * (1 - p)))
geometric_normalized = normalize_means(geometric_means, 1 / p, np.sqrt((1 - p) / p**2))
poisson_normalized = normalize_means(poisson_means, lambda_, np.sqrt(lambda_))

x = np.linspace(-4, 4, 1000)
standard_normal = norm.pdf(x, 0, 1)
hist_range = (-4, 4)

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Convergence of normalized distribution to standard normal')

axs[0].hist(binomial_normalized, bins=30, density=True, alpha=0.6, range=hist_range, label='Normalized Means')
axs[0].plot(x, standard_normal, 'r', label='Standard Normal')
axs[0].set_title('Binomial Distribution')
axs[0].legend()

axs[1].hist(geometric_normalized, bins=30, density=True, alpha=0.6, range=hist_range, label='Normalized Means')
axs[1].plot(x, standard_normal, 'r', label='Standard Normal')
axs[1].set_title('Geometric Distribution')
axs[1].legend()

axs[2].hist(poisson_normalized, bins=30, density=True, alpha=0.6, range=hist_range, label='Normalized Means')
axs[2].plot(x, standard_normal, 'r', label='Standard Normal')
axs[2].set_title('Poisson Distribution')
axs[2].legend()

plt.tight_layout()
plt.show()

## 4. References

1. Introduction to Probability, Statistics and Random Processes: https://probabilitycourse.com/

2. The Book of Statistical Proofs: https://statproofbook.github.io/I/ToC

3. The Art of Computer Programming (Volume 2, Chapter 3): https://www.amazon.in/Art-Computer-Programming-Seminumerical-Algorithms-ebook/dp/B01AY4ZHKE